# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,at taj,24.2049,23.2857,76.84,35,100,7.70,LY,1696214682
1,1,stanley,54.8680,-1.6985,53.78,93,100,4.41,GB,1696214683
2,2,iqaluit,63.7506,-68.5145,37.13,93,100,4.61,CA,1696214556
3,3,ushuaia,-54.8000,-68.3000,35.26,75,40,19.57,AR,1696214465
4,4,waitangi,-43.9535,-176.5597,58.82,87,77,5.99,NZ,1696214683


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

# Code tweaked by askBCS Learning Assistant

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define your criteria for ideal weather conditions in Fahrenheit
max_temp_condition_F = (city_data_df["Max Temp"] >= 60) & (city_data_df["Max Temp"] <= 80)
humidity_condition = (city_data_df["Humidity"] >= 30) & (city_data_df["Humidity"] <= 60) 
wind_speed_condition = city_data_df["Wind Speed"] <= 10
cloudiness_condition = city_data_df["Cloudiness"] <= 50


# Apply the conditions to filter the DataFrame
ideal_weather_df_F = city_data_df[max_temp_condition_F & humidity_condition & wind_speed_condition & cloudiness_condition].copy()

# Drop any rows with null values if necessary
ideal_weather_df_F.dropna(inplace=True)

# Display sample data
ideal_weather_df_F.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,gargalianoi,37.0667,21.6333,66.36,57,4,3.74,GR,1696214514
45,45,barentu,15.1139,37.5928,73.27,57,0,3.31,ER,1696214687
46,46,buriti bravo,-5.8372,-43.8336,77.41,36,16,2.73,BR,1696214687
79,79,portland,45.5234,-122.6762,64.99,60,34,2.42,US,1696214601
139,139,huelma,37.6713,-3.3900,67.98,39,0,4.68,ES,1696214695


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df_F[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
21,gargalianoi,GR,37.0667,21.6333,57,
45,barentu,ER,15.1139,37.5928,57,
46,buriti bravo,BR,-5.8372,-43.8336,36,
79,portland,US,45.5234,-122.6762,60,
139,huelma,ES,37.6713,-3.3900,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius=10000
params = {
    "categories": "accommodation.hotel", # Search for lodging places
    "filter": "circle: radius",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Print a message to follow up the hotel search
    print("End of hotel search")
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

#AskBCS Learning Assistant helped to clarify and tweak the params

Starting hotel search
End of hotel search
gargalianoi - nearest hotel: Messinian Horizons
End of hotel search
barentu - nearest hotel: No hotel found
End of hotel search
buriti bravo - nearest hotel: No hotel found
End of hotel search
portland - nearest hotel: the Hoxton
End of hotel search
huelma - nearest hotel: Sol de Mágina
End of hotel search
hakui - nearest hotel: ビジネスホテル羽音碧々
End of hotel search
argo - nearest hotel: Apollon Hotel
End of hotel search
buka - nearest hotel: No hotel found
End of hotel search
ghat - nearest hotel: فندق تاسيلي
End of hotel search
urangan - nearest hotel: Ramada
End of hotel search
saint-pierre - nearest hotel: Tropic Hotel
End of hotel search
estherville - nearest hotel: No hotel found
End of hotel search
sparti - nearest hotel: Maniatis
End of hotel search
camiri - nearest hotel: Sol Chaqueño
End of hotel search
yamada - nearest hotel: No hotel found
End of hotel search
wauchope - nearest hotel: No hotel found
End of hotel search
serdar - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
21,gargalianoi,GR,37.0667,21.6333,57,Messinian Horizons
45,barentu,ER,15.1139,37.5928,57,No hotel found
46,buriti bravo,BR,-5.8372,-43.8336,36,No hotel found
79,portland,US,45.5234,-122.6762,60,the Hoxton
139,huelma,ES,37.6713,-3.3900,39,Sol de Mágina


In [7]:
# Filter the DataFrame to remove rows where "Hotel Name" is "No hotel found"
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Reset the index of the DataFrame
hotel_df.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,gargalianoi,GR,37.0667,21.6333,57,Messinian Horizons
1,portland,US,45.5234,-122.6762,60,the Hoxton
2,huelma,ES,37.6713,-3.3900,39,Sol de Mágina
3,hakui,JP,36.8833,136.7833,42,ビジネスホテル羽音碧々
4,argo,GR,37.6333,22.7333,53,Apollon Hotel
5,ghat,LY,24.9647,10.1728,39,فندق تاسيلي
6,urangan,AU,-25.3000,152.9000,54,Ramada
7,saint-pierre,RE,-21.3393,55.4781,53,Tropic Hotel
8,sparti,GR,37.0733,22.4297,60,Maniatis
9,camiri,BO,-20.0500,-63.5167,55,Sol Chaqueño


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot for the hotels
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)